In [0]:
from pyspark.sql import functions as F

silver_df = spark.table("retail_lakehouse.silver_sales")


In [0]:
gold_daily_revenue = (
    silver_df
    .groupBy("invoice_date")
    .agg(
        F.sum("total_amount").alias("daily_revenue"),
        F.countDistinct("InvoiceNo").alias("orders"),
        F.countDistinct("CustomerID").alias("unique_customers")
    )
    .orderBy("invoice_date")
)


In [0]:
(
    gold_daily_revenue.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("retail_lakehouse.gold_daily_revenue")
)


In [0]:
gold_top_products = (
    silver_df
    .groupBy("StockCode", "Description")
    .agg(
        F.sum("total_amount").alias("product_revenue"),
        F.sum("Quantity").alias("units_sold")
    )
    .orderBy(F.desc("product_revenue"))
)


In [0]:
(
    gold_top_products.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("retail_lakehouse.gold_top_products")
)


In [0]:
gold_top_customers = (
    silver_df
    .groupBy("CustomerID")
    .agg(
        F.sum("total_amount").alias("customer_revenue"),
        F.countDistinct("InvoiceNo").alias("orders")
    )
    .orderBy(F.desc("customer_revenue"))
)


In [0]:
(
    gold_top_customers.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("retail_lakehouse.gold_top_customers")
)


In [0]:
gold_country_revenue = (
    silver_df
    .groupBy("Country")
    .agg(
        F.sum("total_amount").alias("country_revenue"),
        F.countDistinct("InvoiceNo").alias("orders")
    )
    .orderBy(F.desc("country_revenue"))
)


In [0]:
(
    gold_country_revenue.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("retail_lakehouse.gold_country_revenue")
)


In [0]:
%sql
SELECT COUNT(*) FROM retail_lakehouse.gold_daily_revenue;


COUNT(*)
305


In [0]:
%sql
SELECT * 
FROM retail_lakehouse.gold_top_products 
ORDER BY product_revenue DESC 
LIMIT 10;


StockCode,Description,product_revenue,units_sold
23843.0,"PAPER CRAFT , LITTLE BIRDIE",168469.6,80995
22423.0,REGENCY CAKESTAND 3 TIER,141945.9999999997,12349
85123A,WHITE HANGING HEART T-LIGHT HOLDER,100046.949999999,36589
85099B,JUMBO BAG RED RETROSPOT,84962.28000000126,46040
23166.0,MEDIUM CERAMIC TOP STORAGE JAR,81405.48,77907
POST,POSTAGE,77803.96,3120
47566.0,PARTY BUNTING,68614.98000000032,15244
84879.0,ASSORTED COLOUR BIRD ORNAMENT,56191.63999999953,35132
23084.0,RABBIT NIGHT LIGHT,51070.28000000005,27066
79321.0,CHILLI LIGHTS,46235.40999999998,9640


In [0]:
%sql
SELECT * 
FROM retail_lakehouse.gold_top_customers 
ORDER BY customer_revenue DESC 
LIMIT 10;


CustomerID,customer_revenue,orders
14646,280206.0199999999,73
18102,259657.30000000005,60
17450,194390.78999999998,46
16446,168472.5,2
14911,143711.17000000007,201
12415,124914.53000000003,21
14156,117193.58000000003,55
17511,91062.37999999998,31
16029,80613.23999999999,63
12346,77183.6,1
